<a href="https://colab.research.google.com/github/buun1030/SG-Train/blob/main/participants_SGx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# เชื่อมต่อ Google colab กับ Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install pandasql
import pandas as pd
import pandasql as ps
import numpy as np
import re
from google.colab import files

In [ ]:
AB = pd.read_csv('/content/drive/MyDrive/agency_list.csv')
a = AB['A'].astype(str).values.tolist()
b = AB['B'].astype(str).values.tolist()
c = ['sgteam','sg team','พรรณราย','อรรณพ','โสภณ','อรณี','บรมัตถ์','พัชรินทร์ เยาวรัตน์','สันต์',\
     'annop unn','oranee','boramat','phatcharin yaowarat','sunt','prapita','icbrg',\
     'sgtech','นิพนธ์ เกตุจ้อย','ยอดธง เม่นสิน','วสันต์ ติรางกูร',\
     'nipon','yodthong','vazzan','tawat','wisut','malinee kaewpanha','meeting','admin',\
     'puckpimon singhapong','lu zhao','zibo dong','puckpimon']

In [ ]:
file_name = 'participants-SG7'

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/' + file_name + '.csv')

In [ ]:
df.info()

In [ ]:
# use this when number of columns is over
# df.drop(df.columns[[7, 8, 9, 10, 11, 12, 13]], axis = 1, inplace = True)

In [ ]:
# create agency column with correspond agency in each row
df['Agency'] = ''
row = 0
for x in df['Name (Original Name)']:
  for i in range(len(a)):
    if a[i].upper() in x.upper():
      df['Agency'][row] = b[i].upper()
  row += 1

# fill empty cells with 'Other'
df['Agency'][df['Agency'] == ''] = 'Other'

In [ ]:
# select needed columns 
df_forCount = df[['Name (Original Name)','Duration (Minutes)','Agency']]
print('Original')
print(df_forCount.count())
print()

# filter rows that duration > 50 minutes
duration = 50
df_forCount = df_forCount[df['Duration (Minutes)'] > duration]
print('After filter rows that duration > ' + str(duration) + ' minutes')
print(df_forCount.count())
print()
df_forCount = df_forCount.reset_index().drop(labels='index', axis=1)

d = []
row = 0
for x in df_forCount['Name (Original Name)']:
  for d_name in c:
    if d_name.upper() in x.upper():
      d.append(row)
  row += 1
d = list(dict.fromkeys(d))

# drop rows with d index
df_forCount = df_forCount.drop(labels=d, axis=0)
print('After drop rows with staff name')
print(df_forCount.count())
print()

# drop duplicate rows
df_forCount = df_forCount.drop_duplicates(subset = ['Name (Original Name)'])
print('After drop duplicate rows')
print(df_forCount.count())

In [ ]:
# check Agency list
df_forCount['Agency'].sort_values().unique()

In [ ]:
# display all rows
pd.set_option('display.max_rows', None)
# display full (non-truncated) dataframe information
pd.set_option('display.max_colwidth', None)

In [ ]:
# manually correct wrong Agency
df_forCount['Agency'][df_forCount['Name (Original Name)']=='Kmutnb Palika Rattanaburee']='KMUTNB'

In [ ]:
# manually delete duplicate Name
df_forCount = df_forCount.drop(df_forCount[df_forCount['Name (Original Name)'] == 'Poch'].index)

In [ ]:
# manually check duplicate Name
ps.sqldf('SELECT * FROM df_forCount ORDER BY [Agency], [Name (Original Name)]')

In [ ]:
# pivot
df_pivot = pd.pivot_table(df_forCount, values='Name (Original Name)', index=['Agency'], aggfunc='count', margins = True, fill_value=0)
df_pivot

In [ ]:
# save & download to excel
writer = pd.ExcelWriter(file_name + '_final.xlsx')

df.to_excel(writer, sheet_name = 'par', index = False)
df_forCount.to_excel(writer, sheet_name = 'par_forCount', index = False)
df_pivot.to_excel(writer, sheet_name = 'pivot', index = True)

writer.save()
files.download(file_name + '_final.xlsx')
# All done here!

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>